In [26]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
import random
from glob import glob
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

from transformers import AutoProcessor, AutoModel
from peft import get_peft_model, LoraConfig

warnings.filterwarnings(action="ignore")
TARGET_LABEL = 'Pneumonia'

def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

!pip install -q pydicom
from glob import glob
import pydicom

DATA_PATH = 'data'
MIMIC_DATA_DIR = os.path.join(DATA_PATH, 'MIMIC-CXR-Data')
IMAGE_BASE_PATH = os.path.join(MIMIC_DATA_DIR, 'MIMIC-CXR')

if not os.path.exists(IMAGE_BASE_PATH):
    print("Downloading and unzipping MIMIC-CXR data...")
    !wget https://uni-bonn.sciebo.de/s/YHuwFOg6q6sw1ZX/download -O {DATA_PATH}/MIMIC-CXR.zip
    !unzip -q -o {DATA_PATH}/MIMIC-CXR.zip -d {MIMIC_DATA_DIR}

try:

    record_df = pd.read_csv(os.path.join(IMAGE_BASE_PATH, 'cxr-record-list.csv.gz'))
    labels_df = pd.read_csv(os.path.join(IMAGE_BASE_PATH, 'cxr-negbio-list.csv.gz'))


    print("Searching for available .dcm files on disk...")
    file_list = glob(f'{IMAGE_BASE_PATH}/files/**/*.dcm', recursive=True)
    relative_file_list = [os.path.relpath(p, IMAGE_BASE_PATH) for p in file_list]
    print(f"Found {len(relative_file_list)} existing .dcm image files.")


    master_df = record_df[record_df['path'].isin(relative_file_list)].copy()
    if len(master_df) == 0:
        raise ValueError("The glob search found no .dcm files, or paths do not match.")


    master_df = pd.merge(master_df, labels_df, on=['subject_id', 'study_id'], how='inner')


    master_df['Patient Age'] = np.random.randint(40, 85, size=len(master_df))
    master_df['Patient Sex'] = np.random.choice(['F', 'M'], size=len(master_df))
    master_df['ICU Stay'] = np.random.randint(0, 2, size=len(master_df))
    master_df = master_df.rename(columns={'path': 'ImagePath'})

    master_df[TARGET_LABEL] = master_df[TARGET_LABEL].fillna(0).astype(int)

    required_cols = ['ImagePath', 'Patient Age', 'Patient Sex', 'ICU Stay', TARGET_LABEL]
    final_df = master_df[required_cols].dropna().reset_index(drop=True)


    train_val_df, test_df = train_test_split(final_df, test_size=0.1, random_state=42)
    train_df, val_df = train_test_split(train_val_df, test_size=0.1, random_state=42)

    IMG_DIR = IMAGE_BASE_PATH

    print("MIMIC-CXR data loaded and successfully filtered.")
    print(f"Total usable samples: {len(final_df)}")
    print(f"Training samples: {len(train_df)}")
    print(f"Validation samples: {len(val_df)}")

except Exception as e:
    print(f"An error occurred. Details: {e}")
    train_df, val_df, test_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

Using device: cpu
Searching for available .dcm files on disk...
Found 1033 existing .dcm image files.
MIMIC-CXR data loaded and successfully filtered.
Total usable samples: 1033
Training samples: 836
Validation samples: 93


In [27]:

import pydicom

def read_dicom_image(path):
    try:
        dcm = pydicom.dcmread(path)
        arr = dcm.pixel_array
        arr = arr.astype(np.float32)
        if hasattr(dcm, 'PhotometricInterpretation') and dcm.PhotometricInterpretation == "MONOCHROME1":
            arr = np.amax(arr) - arr
        arr = (arr - arr.min()) / (arr.max() - arr.min() + 1e-6) * 255.0
        arr = arr.astype(np.uint8)
        return Image.fromarray(arr)
    except Exception:
        return Image.new('L', (224, 224), 0)

if not train_df.empty:
    min_age = train_df['Patient Age'].min()
    max_age = train_df['Patient Age'].max()
    def preprocess_tabular_mimic(age, sex, icu_stay):
        age_norm = (age - min_age) / (max_age - min_age) if max_age > min_age else 0
        sex_onehot = [1.0, 0.0] if sex == 'F' else [0.0, 1.0]
        icu_feature = [float(icu_stay)]
        return torch.tensor([age_norm] + sex_onehot + icu_feature, dtype=torch.float32)
    TABULAR_FEATURE_SIZE = len(preprocess_tabular_mimic(60, 'F', 1))
else:
    TABULAR_FEATURE_SIZE = 4

image_size = 224
train_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
val_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

class MIMICScratchDataset(Dataset):
    def __init__(self, df, img_dir, transform):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        full_path = os.path.join(self.img_dir, row['ImagePath'])
        image = read_dicom_image(full_path).convert('RGB')
        image = self.transform(image)
        tabular = preprocess_tabular_mimic(row['Patient Age'], row['Patient Sex'], row['ICU Stay'])
        label = torch.tensor(row[TARGET_LABEL], dtype=torch.float32)
        return image, tabular, label

class MultiModalFMDataset(Dataset):
    def __init__(self, df, img_dir, processor):
        self.df = df
        self.img_dir = img_dir
        self.processor = processor
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        full_path = os.path.join(self.img_dir, row['ImagePath'])
        image = read_dicom_image(full_path).convert('RGB')
        image_inputs = self.processor(images=image, return_tensors="pt")
        image_tensor = image_inputs['pixel_values'].squeeze(0)
        tabular_tensor = preprocess_tabular_mimic(row['Patient Age'], row['Patient Sex'], row['ICU Stay'])
        label = torch.tensor(row[TARGET_LABEL], dtype=torch.float32)
        return image_tensor, tabular_tensor, label

if not train_df.empty:
    processor = AutoProcessor.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32")
    train_ds_scratch = MIMICScratchDataset(train_df, IMG_DIR, train_transforms)
    val_ds_scratch = MIMICScratchDataset(val_df, IMG_DIR, val_transforms)
    train_loader_scratch = DataLoader(train_ds_scratch, batch_size=32, shuffle=True, num_workers=2)
    val_loader_scratch = DataLoader(val_ds_scratch, batch_size=32, shuffle=False, num_workers=2)
    train_ds_fm = MultiModalFMDataset(train_df, IMG_DIR, processor)
    val_ds_fm = MultiModalFMDataset(val_df, IMG_DIR, processor)
    train_loader_fm = DataLoader(train_ds_fm, batch_size=32, shuffle=True, num_workers=2)
    val_loader_fm = DataLoader(val_ds_fm, batch_size=32, shuffle=False, num_workers=2)
    print("\nDataLoaders are ready.")


DataLoaders are ready.


In [28]:
class ConcatenationModel(nn.Module):
    def __init__(self, num_tabular_features):
        super().__init__()
        self.image_encoder = models.resnet18(pretrained=True)
        num_image_features = self.image_encoder.fc.in_features
        self.image_encoder.fc = nn.Identity()
        self.classifier = nn.Sequential(
            nn.Linear(num_image_features + num_tabular_features, 256),
            nn.ReLU(), nn.Dropout(0.3), nn.Linear(256, 1)
        )
    def forward(self, image, tabular):
        image_features = self.image_encoder(image)
        combined_features = torch.cat([image_features, tabular], dim=1)
        return self.classifier(combined_features)

class CrossAttentionModel(nn.Module):
    def __init__(self, num_tabular_features):
        super().__init__()
        self.image_encoder = models.resnet18(pretrained=True)
        num_image_features = self.image_encoder.fc.in_features
        self.image_feature_extractor = nn.Sequential(*list(self.image_encoder.children())[:-2])
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=num_tabular_features, num_heads=4,
            kdim=num_image_features, vdim=num_image_features, batch_first=True
        )
        self.classifier = nn.Sequential(
            nn.Linear(num_tabular_features, 128), nn.ReLU(), nn.Linear(128, 1)
        )
    def forward(self, image, tabular):
        image_patches = self.image_feature_extractor(image)
        b, c, h, w = image_patches.shape
        image_patches = image_patches.view(b, c, -1).permute(0, 2, 1)
        query = tabular.unsqueeze(1)
        fused_features, _ = self.cross_attention(query, image_patches, image_patches)
        return self.classifier(fused_features.squeeze(1))

In [29]:
medclip_model_base = AutoModel.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32")
IMAGE_EMBED_DIM = medclip_model_base.visual_projection.out_features

class LinearProbeModel(nn.Module):
    def __init__(self, vision_model, num_img_features, num_tabular_features):
        super().__init__()
        self.vision_model = vision_model
        for param in self.vision_model.parameters(): param.requires_grad = False
        self.classifier = nn.Linear(num_img_features + num_tabular_features, 1)
    def forward(self, image, tabular):
        with torch.no_grad():
            img_features = self.vision_model.get_image_features(pixel_values=image)
        return self.classifier(torch.cat([img_features, tabular], dim=1))

class PartialFTModel(nn.Module):
    def __init__(self, vision_model, num_img_features, num_tabular_features):
        super().__init__()
        self.vision_model = vision_model
        for param in self.vision_model.parameters(): param.requires_grad = False
        self.classifier = nn.Sequential(
            nn.Linear(num_img_features + num_tabular_features, 128),
            nn.ReLU(), nn.Dropout(0.3), nn.Linear(128, 1)
        )
    def forward(self, image, tabular):
        with torch.no_grad():
            img_features = self.vision_model.get_image_features(pixel_values=image)
        return self.classifier(torch.cat([img_features, tabular], dim=1))

class LoRAModel(nn.Module):
    def __init__(self, vision_model, num_img_features, num_tabular_features):
        super().__init__()
        for param in vision_model.parameters(): param.requires_grad = False
        lora_config = LoraConfig(r=16, lora_alpha=16, lora_dropout=0.1, bias="none", target_modules=["q_proj", "v_proj"])
        self.vision_model = get_peft_model(vision_model, lora_config)
        self.classifier = nn.Sequential(
            nn.Linear(num_img_features + num_tabular_features, 128),
            nn.ReLU(), nn.Dropout(0.3), nn.Linear(128, 1)
        )
    def forward(self, image, tabular):
        img_features = self.vision_model.get_image_features(pixel_values=image)
        return self.classifier(torch.cat([img_features, tabular], dim=1))

In [39]:
def train_and_evaluate(model, model_name, train_loader, val_loader, epochs=5, debug_first_epoch=True):
    print(f"Training {model_name}")
    model.to(device)
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
    criterion = nn.BCEWithLogitsLoss()
    best_val_auc, best_val_acc, best_val_f1 = 0.0, 0.0, 0.0
    did_debug = False

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for data in tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False):
            images, tabular, labels = data[0].to(device), data[1].to(device), data[2].to(device)
            optimizer.zero_grad()
            outputs = model(images, tabular)
            loss = criterion(outputs.squeeze(-1), labels)
            loss.backward()
            optimizer.step()
            total_loss += float(loss.item())


        model.eval()
        all_labels, all_probs = [], []
        with torch.no_grad():
            for data in val_loader:
                images, tabular, labels = data[0].to(device), data[1].to(device), data[2].to(device)
                outputs = model(images, tabular)


                probs = torch.sigmoid(outputs.squeeze(-1))
                probs = probs.cpu().numpy()
                probs = np.nan_to_num(probs, nan=0.5, posinf=1.0, neginf=0.0)

                all_probs.extend(probs.ravel())
                all_labels.extend(labels.cpu().numpy().ravel())


        all_labels = np.asarray(all_labels)
        all_probs = np.asarray(all_probs)



        if all_labels.size == 0:

            val_auc, val_acc, val_f1 = 0.0, 0.0, 0.0
        else:
            unique_labels = np.unique(all_labels)


            val_auc = 0.5
            try:

                if all_probs.ndim == 2 and all_probs.shape[1] > 1 and len(unique_labels) > 2:
                    val_auc = roc_auc_score(all_labels, all_probs, multi_class='ovr', average='macro')
                else:

                    if len(unique_labels) <= 2:
                        val_auc = roc_auc_score(all_labels, all_probs)
                    else:

                        val_auc = 0.5
            except Exception as e:

                val_auc = 0.5


            try:
                val_preds = (all_probs > 0.5).astype(int)
            except Exception:

                if all_probs.ndim == 2 and all_probs.shape[1] > 1:
                    val_preds = np.argmax(all_probs, axis=1)
                else:
                    val_preds = np.zeros_like(all_labels, dtype=int)

            val_acc = accuracy_score(all_labels, val_preds)


            if len(unique_labels) <= 2:
                val_f1 = f1_score(all_labels, val_preds, average='binary', zero_division=0)
            else:

                val_f1 = f1_score(all_labels, val_preds, average='weighted', zero_division=0)

        print(f"Epoch {epoch+1} | Loss: {total_loss/max(1,len(train_loader)):.4f} | Val AUC: {val_auc:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}")


        if val_auc > best_val_auc: best_val_auc = val_auc
        if val_acc > best_val_acc: best_val_acc = val_acc
        if val_f1 > best_val_f1: best_val_f1 = val_f1

    return {'name': model_name, 'auc': best_val_auc, 'acc': best_val_acc, 'f1': best_val_f1,
            'params': sum(p.numel() for p in model.parameters() if p.requires_grad)}


In [40]:

models_to_run = [
    ("Concatenation (Scratch)", lambda: ConcatenationModel(num_tabular_features=TABULAR_FEATURE_SIZE), train_loader_scratch, val_loader_scratch),
    ("Cross-Attention (Scratch)", lambda: CrossAttentionModel(num_tabular_features=TABULAR_FEATURE_SIZE), train_loader_scratch, val_loader_scratch),
    ("Linear Probing (MedCLIP)", lambda: LinearProbeModel(AutoModel.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32"), IMAGE_EMBED_DIM, TABULAR_FEATURE_SIZE), train_loader_fm, val_loader_fm),
    ("Partial FT (MedCLIP)", lambda: PartialFTModel(AutoModel.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32"), IMAGE_EMBED_DIM, TABULAR_FEATURE_SIZE), train_loader_fm, val_loader_fm),
    ("LoRA (MedCLIP)", lambda: LoRAModel(AutoModel.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32"), IMAGE_EMBED_DIM, TABULAR_FEATURE_SIZE), train_loader_fm, val_loader_fm),
]

results = []

for name, model_factory, tr_loader, vl_loader in models_to_run:
    print(f"\n=> Starting: {name}")
    try:
        model = model_factory()
        res = train_and_evaluate(model, name, tr_loader, vl_loader, epochs=5)
        results.append(res)
        print(f"Completed : {name} -> {res}")
    except Exception as e:

        print(f"ERROR during training  {name}: {e}")
        results.append({'name': name, 'auc': np.nan, 'acc': np.nan, 'f1': np.nan, 'params': 0})



=> Starting: Concatenation (Scratch)
--- Training Concatenation (Scratch) ---


Epoch 1:   0%|          | 0/27 [00:00<?, ?it/s]

DEBUG (first val): unique labels: [-1.  0.  1.]
DEBUG (first val): all_probs shape: (93,)
Epoch 1 | Loss: 0.0772 | Val AUC: 0.5000 | Acc: 0.8817 | F1: 0.8263


Epoch 2:   0%|          | 0/27 [00:00<?, ?it/s]

Epoch 2 | Loss: -0.5926 | Val AUC: 0.5000 | Acc: 0.8817 | F1: 0.8263


...

Epoch 3 | Loss: 0.0011 | Val AUC: 0.5000 | Acc: 0.8817 | F1: 0.8263


Epoch 4:   0%|          | 0/27 [00:00<?, ?it/s]

Epoch 4 | Loss: -0.0450 | Val AUC: 0.5000 | Acc: 0.8817 | F1: 0.8263


Epoch 5:   0%|          | 0/27 [00:00<?, ?it/s]

Epoch 5 | Loss: -0.1212 | Val AUC: 0.5000 | Acc: 0.8817 | F1: 0.8263
Completed: LoRA (MedCLIP) -> {'name': 'LoRA (MedCLIP)', 'auc': 0.5, 'acc': 0.8817204301075269, 'f1': 0.8262980030721967, 'params': 1049345}
